# DB Systel MLOps Workshop

## Zusammenfassung der Befehle des Workshops

### 1 Einrichtung der Sandbox

Folgt der Anleitung in `readme.md` zum erstellen der Sandbox Umgebung.

### 2 Konfiguration unserer MLOps Sandbox

#### 2.1 Installiert Flyte in eurem k3d Cluster

Fügt das `flyteorg` helm repository hinzu. 

(erfordert die Installation des Kubernetes Package Managers `Helm`, führt dazu `brew install helm` oder siehe die Installationsseite, [hier](https://helm.sh/docs/intro/install/))


In [1]:
!helm repo add flyteorg https://helm.flyte.org

"flyteorg" has been added to your repositories


Installiert die flyte dependencies, `flyte-deps`

In [4]:
!helm install flyte-deps flyteorg/flyte-deps -n flyte --create-namespace -f https://raw.githubusercontent.com/flyteorg/flyte/master/charts/flyte-deps/values-sandbox.yaml

W0930 09:40:48.256859   95921 warnings.go:70] spec.template.metadata.annotations[seccomp.security.alpha.kubernetes.io/pod]: deprecated since v1.19, non-functional in v1.25+; use the "seccompProfile" field instead
NAME: flyte-deps
LAST DEPLOYED: Fri Sep 30 09:39:47 2022
NAMESPACE: flyte
STATUS: deployed
REVISION: 1
TEST SUITE: None


Hier haben wir einen `Minio Bucket` (ähnlich wie S3), `postgres`, das `Flyte Dashboard` und einen `contour` ingress controller installiert.

Installiert `flyte-core`

In [5]:
!helm install flyte flyteorg/flyte-core -n flyte -f https://raw.githubusercontent.com/flyteorg/flyte/master/charts/flyte-core/values-sandbox.yaml --wait

NAME: flyte
LAST DEPLOYED: Fri Sep 30 09:47:04 2022
NAMESPACE: flyte
STATUS: deployed
REVISION: 1
TEST SUITE: None


Überzeugt euch, dass die Pods da sind

In [6]:
!kubectl get pods -n flyte

NAME                                               READY   STATUS    RESTARTS   AGE
minio-688d8dc56d-v67mb                             1/1     Running   0          7m38s
flyte-deps-kubernetes-dashboard-77b9d9cdc7-gmcq5   1/1     Running   0          7m38s
postgres-85847b7786-7k9xc                          1/1     Running   0          7m38s
flyte-deps-contour-contour-57d8b8b6f5-mfg6b        1/1     Running   0          7m38s
flyte-deps-contour-envoy-jbd62                     2/2     Running   0          7m38s
datacatalog-8686dfcbcb-j77zx                       1/1     Running   0          81s
flyteconsole-7698b79994-cmkpq                      1/1     Running   0          81s
syncresources-5cd85b565f-j5wfp                     1/1     Running   0          81s
flytepropeller-585647b49-ph55c                     1/1     Running   0          81s
flyteadmin-7dfb47f7bc-rl4lr                        1/1     Running   0          81s
flytescheduler-5d5fcdf84c-zhr5h                    1/1     Running

`-n flyte` steht für `--namespace flyte`.

Öffnet nun unter http://127.0.0.1:30081/console/ das Flyte Dashboard im Browser.

**NOTE** Wir hatten den k3d Cluster so konfiguriert, das manche Ports falls möglich automatisch auf localhost verfügbar gemacht werden. Falls der Port 30081 nicht verfügbar ist müssen wir diesen manuell in einem anderen Terminalfenster verfügbar machen: 

```bash
kubectl port-forward svc/flyte-deps-contour-envoy 30081:80 -n flyte
```

Dieser Befehlt leitet localhost:30081 auf Port 80 des Service `flyte-deps-contour-envoy` weiter, welcher sich um die Weiterleitung zum Flyte Dashboard kümmert.

### 3 Pipeline packagen, registrieren und ausführen

#### 3.1 Docker image bauen und pushen

Unser Flyte Workflow benötigt ein Image in dem u.A. alle Dependencies enthalten sind, welche in den Flyte Workflow importiert werden. Wir bauen dieses Image mit dem Dockerfile `./Dockerfile`

In [7]:
!docker build -t localhost:5000/workflow:latest .

[+] Building 0.0s (0/1)                                                         
[+] Building 0.1s (2/2)                                                         
 => [internal] load build definition from Dockerfile                       0.1s
 => => transferring dockerfile: 976B                                       0.1s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 2B                                            0.0s
[+] Building 0.3s (2/3)                                                         
 => [internal] load build definition from Dockerfile                       0.1s
 => => transferring dockerfile: 976B                                       0.1s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 2B                                            0.0s
 => [internal] load metadata for docker.io/library/python:3.8-slim-buster  0.1s
[+] Building 0.4s (2/3)              

[+] Building 2.3s (4/14)                                                        
 => [internal] load build definition from Dockerfile                       0.1s
 => => transferring dockerfile: 976B                                       0.1s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 2B                                            0.0s
 => [internal] load metadata for docker.io/library/python:3.8-slim-buster  1.9s
 => [ 1/10] FROM docker.io/library/python:3.8-slim-buster@sha256:8950916b  0.3s
 => => resolve docker.io/library/python:3.8-slim-buster@sha256:8950916b06  0.0s
 => => sha256:8950916b06e83a260135f49e77aef6b2019c7f5c7a8ffb7 988B / 988B  0.0s
 => => sha256:2ed874c61a3662aa0e7a97d32f88598ad7dadd9f7c4 1.37kB / 1.37kB  0.0s
 => => sha256:29d82668a2933e3c22323d80f75f24d8e693957234c 7.55kB / 7.55kB  0.0s
 => => sha256:4c3f5cce277263c7aeaf67f83255af76927e03363e775f 0B / 25.90MB  0.2s
 => => sha256:a89d6575799a6c759ac9c7977

 => => sha256:2ed874c61a3662aa0e7a97d32f88598ad7dadd9f7c4 1.37kB / 1.37kB  0.0s
 => => sha256:29d82668a2933e3c22323d80f75f24d8e693957234c 7.55kB / 7.55kB  0.0s
 => => sha256:4c3f5cce277263c7aeaf67f83255af76927e03363 13.63MB / 25.90MB  0.9s
 => => sha256:a89d6575799a6c759ac9c797715af9c00143b68aaa1 2.64MB / 2.64MB  0.6s
 => => sha256:f84a0b5faac35827f6db40b236bdbb5ef36f7ec969 4.19MB / 11.36MB  0.9s
 => => sha256:7d70b2d9c6f67dfa73b13cf9ebfdd5a91fc755de13e7b5e 232B / 232B  0.8s
 => => sha256:4f1e2ff83b6164b6f28d6155ccec261d81306401cd2f13a 0B / 2.96MB  0.9s
 => [internal] load build context                                          0.0s
 => => transferring context: 248B                                          0.0s
[+] Building 3.1s (4/14)                                                        
 => [internal] load build definition from Dockerfile                       0.1s
 => => transferring dockerfile: 976B                                       0.1s
 => [internal] load .dockerignore      

[+] Building 3.7s (4/14)                                                        
 => [internal] load build definition from Dockerfile                       0.1s
 => => transferring dockerfile: 976B                                       0.1s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 2B                                            0.0s
 => [internal] load metadata for docker.io/library/python:3.8-slim-buster  1.9s
 => [ 1/10] FROM docker.io/library/python:3.8-slim-buster@sha256:8950916b  1.7s
 => => resolve docker.io/library/python:3.8-slim-buster@sha256:8950916b06  0.0s
 => => sha256:8950916b06e83a260135f49e77aef6b2019c7f5c7a8ffb7 988B / 988B  0.0s
 => => sha256:2ed874c61a3662aa0e7a97d32f88598ad7dadd9f7c4 1.37kB / 1.37kB  0.0s
 => => sha256:29d82668a2933e3c22323d80f75f24d8e693957234c 7.55kB / 7.55kB  0.0s
 => => sha256:4c3f5cce277263c7aeaf67f83255af76927e03363 23.07MB / 25.90MB  1.6s
 => => sha256:a89d6575799a6c759ac9c7977

[+] Building 4.3s (4/14)                                                        
 => [internal] load build definition from Dockerfile                       0.1s
 => => transferring dockerfile: 976B                                       0.1s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 2B                                            0.0s
 => [internal] load metadata for docker.io/library/python:3.8-slim-buster  1.9s
 => [ 1/10] FROM docker.io/library/python:3.8-slim-buster@sha256:8950916b  2.3s
 => => resolve docker.io/library/python:3.8-slim-buster@sha256:8950916b06  0.0s
 => => sha256:8950916b06e83a260135f49e77aef6b2019c7f5c7a8ffb7 988B / 988B  0.0s
 => => sha256:2ed874c61a3662aa0e7a97d32f88598ad7dadd9f7c4 1.37kB / 1.37kB  0.0s
 => => sha256:29d82668a2933e3c22323d80f75f24d8e693957234c 7.55kB / 7.55kB  0.0s
 => => sha256:4c3f5cce277263c7aeaf67f83255af76927e03363 25.90MB / 25.90MB  1.7s
 => => sha256:a89d6575799a6c759ac9c7977

...

[+] Building 62.6s (14/15)                                                      
 => => sha256:a89d6575799a6c759ac9c797715af9c00143b68aaa1 2.64MB / 2.64MB  0.6s
 => => sha256:f84a0b5faac35827f6db40b236bdbb5ef36f7ec96 11.36MB / 11.36MB  1.3s
 => => sha256:7d70b2d9c6f67dfa73b13cf9ebfdd5a91fc755de13e7b5e 232B / 232B  0.8s
 => => sha256:4f1e2ff83b6164b6f28d6155ccec261d81306401cd2 2.96MB / 2.96MB  1.3s
 => => extracting sha256:4c3f5cce277263c7aeaf67f83255af76927e03363e775f39  0.8s
 => => extracting sha256:a89d6575799a6c759ac9c797715af9c00143b68aaa142dea  0.1s
 => => extracting sha256:f84a0b5faac35827f6db40b236bdbb5ef36f7ec969e38c6f  0.3s
 => => extracting sha256:7d70b2d9c6f67dfa73b13cf9ebfdd5a91fc755de13e7b5ee  0.0s
 => => extracting sha256:4f1e2ff83b6164b6f28d6155ccec261d81306401cd2f13ab  0.1s
 => [internal] load build context                                          0.0s
 => => transferring context: 248B                                          0.0s
 => [ 2/10] WORKDIR /root              

[+] Building 63.2s (14/15)                                                      
 => => sha256:a89d6575799a6c759ac9c797715af9c00143b68aaa1 2.64MB / 2.64MB  0.6s
 => => sha256:f84a0b5faac35827f6db40b236bdbb5ef36f7ec96 11.36MB / 11.36MB  1.3s
 => => sha256:7d70b2d9c6f67dfa73b13cf9ebfdd5a91fc755de13e7b5e 232B / 232B  0.8s
 => => sha256:4f1e2ff83b6164b6f28d6155ccec261d81306401cd2 2.96MB / 2.96MB  1.3s
 => => extracting sha256:4c3f5cce277263c7aeaf67f83255af76927e03363e775f39  0.8s
 => => extracting sha256:a89d6575799a6c759ac9c797715af9c00143b68aaa142dea  0.1s
 => => extracting sha256:f84a0b5faac35827f6db40b236bdbb5ef36f7ec969e38c6f  0.3s
 => => extracting sha256:7d70b2d9c6f67dfa73b13cf9ebfdd5a91fc755de13e7b5ee  0.0s
 => => extracting sha256:4f1e2ff83b6164b6f28d6155ccec261d81306401cd2f13ab  0.1s
 => [internal] load build context                                          0.0s
 => => transferring context: 248B                                          0.0s
 => [ 2/10] WORKDIR /root              

[+] Building 63.8s (14/15)                                                      
 => => sha256:a89d6575799a6c759ac9c797715af9c00143b68aaa1 2.64MB / 2.64MB  0.6s
 => => sha256:f84a0b5faac35827f6db40b236bdbb5ef36f7ec96 11.36MB / 11.36MB  1.3s
 => => sha256:7d70b2d9c6f67dfa73b13cf9ebfdd5a91fc755de13e7b5e 232B / 232B  0.8s
 => => sha256:4f1e2ff83b6164b6f28d6155ccec261d81306401cd2 2.96MB / 2.96MB  1.3s
 => => extracting sha256:4c3f5cce277263c7aeaf67f83255af76927e03363e775f39  0.8s
 => => extracting sha256:a89d6575799a6c759ac9c797715af9c00143b68aaa142dea  0.1s
 => => extracting sha256:f84a0b5faac35827f6db40b236bdbb5ef36f7ec969e38c6f  0.3s
 => => extracting sha256:7d70b2d9c6f67dfa73b13cf9ebfdd5a91fc755de13e7b5ee  0.0s
 => => extracting sha256:4f1e2ff83b6164b6f28d6155ccec261d81306401cd2f13ab  0.1s
 => [internal] load build context                                          0.0s
 => => transferring context: 248B                                          0.0s
 => [ 2/10] WORKDIR /root              

[+] Building 64.4s (14/15)                                                      
 => => sha256:a89d6575799a6c759ac9c797715af9c00143b68aaa1 2.64MB / 2.64MB  0.6s
 => => sha256:f84a0b5faac35827f6db40b236bdbb5ef36f7ec96 11.36MB / 11.36MB  1.3s
 => => sha256:7d70b2d9c6f67dfa73b13cf9ebfdd5a91fc755de13e7b5e 232B / 232B  0.8s
 => => sha256:4f1e2ff83b6164b6f28d6155ccec261d81306401cd2 2.96MB / 2.96MB  1.3s
 => => extracting sha256:4c3f5cce277263c7aeaf67f83255af76927e03363e775f39  0.8s
 => => extracting sha256:a89d6575799a6c759ac9c797715af9c00143b68aaa142dea  0.1s
 => => extracting sha256:f84a0b5faac35827f6db40b236bdbb5ef36f7ec969e38c6f  0.3s
 => => extracting sha256:7d70b2d9c6f67dfa73b13cf9ebfdd5a91fc755de13e7b5ee  0.0s
 => => extracting sha256:4f1e2ff83b6164b6f28d6155ccec261d81306401cd2f13ab  0.1s
 => [internal] load build context                                          0.0s
 => => transferring context: 248B                                          0.0s
 => [ 2/10] WORKDIR /root              

[+] Building 64.9s (15/15) FINISHED                                             
 => [internal] load build definition from Dockerfile                       0.1s
 => => transferring dockerfile: 976B                                       0.1s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 2B                                            0.0s
 => [internal] load metadata for docker.io/library/python:3.8-slim-buster  1.9s
 => [ 1/10] FROM docker.io/library/python:3.8-slim-buster@sha256:8950916b  3.3s
 => => resolve docker.io/library/python:3.8-slim-buster@sha256:8950916b06  0.0s
 => => sha256:8950916b06e83a260135f49e77aef6b2019c7f5c7a8ffb7 988B / 988B  0.0s
 => => sha256:2ed874c61a3662aa0e7a97d32f88598ad7dadd9f7c4 1.37kB / 1.37kB  0.0s
 => => sha256:29d82668a2933e3c22323d80f75f24d8e693957234c 7.55kB / 7.55kB  0.0s
 => => sha256:4c3f5cce277263c7aeaf67f83255af76927e03363 25.90MB / 25.90MB  1.7s
 => => sha256:a89d6575799a6c759ac9c7977

In [8]:
!docker push localhost:5000/workflow:latest

The push refers to repository [localhost:5000/workflow]

9759e86d: Preparing 
94251120: Preparing 
afac4dd9: Preparing 
c2d0bb6f: Preparing 
61d49a78: Preparing 
653cbe84: Preparing 
d8036a36: Preparing 
0b283edc: Preparing 
bf18a086: Preparing 
9403e0dd: Preparing 
a7e617d1: Preparing 
7e01783b: Preparing 
ea2ac3ba: Preparing 
759e86d: Pushed   1.278GB/1.255GBPushing  16.02MB/28.52MBlatest: digest: sha256:c744206fda6930585e616973320cb684dae106363b40b0b4a5194665cc56eb18 size: 3260


Schaut euch an was in einem Container mit diesem Image enthalten ist:

In [9]:
!docker run -it localhost:5000/workflow ls

bin  install.sh  requirements.txt


Der eigentliche Workflow code ist im Docker Image noch nicht enthalten. Flyte erstellt bei der Registrierung des Workflows ein tar Archiv, welches in einem Bucket gespeichert wird. Die Worker, die die tasks des Workflows ausführen, laden den Code aus diesem Bucket heruntern. Dies erlaubt schnelles Iterieren bei der Entwicklung des Workflows, da das Docker Image nur erneut gebaut werden muss, wenn neue Requirements hinzugefügt werden.

#### 3.2 Packen und registrieren des Workflows  (`workflow_part1.py`)
Dieser befindet sich in `flytesnacks/workflows/workflow_part1.py` und wird durch das folgende Script registriert:

In [10]:
!pyflyte --config flyte_config register --project flytesnacks\
        --image k3d-registry.localhost:5000/workflow:latest\
        --version 1 flytesnacks/workflows/workflow_part1.py

{"asctime": "2022-09-30 10:13:06,726", "name": "flytekit.cli", "levelname": "WARNING", "message": "Could not determine ignored files due to:\nb'fatal: not a git repository (or any of the parent directories): .git\\n'\nNot applying any filters"}
Output given as None, using a temporary directory at /var/folders/_y/xylsrmns14x1p81tf7bw4jyw0000gn/T/tmpeiekcz_9 instead
{"asctime": "2022-09-30 10:13:07,370", "name": "flytekit.remote", "levelname": "WARNING", "message": "Uploading /var/folders/_y/xylsrmns14x1p81tf7bw4jyw0000gn/T/tmpeiekcz_9/fast6b3186ba5a67b7741f25c9926f203578.tar.gz to http://localhost:30084/my-s3-bucket/m2/flytesnacks/development/RMQEH6CP2T5OI2YGNTNG2KR5CM%3D%3D%3D%3D%3D%3D/fast6b3186ba5a67b7741f25c9926f203578.tar.gz?X-Amz-Algorithm=AWS4-HMAC-SHA256&X-Amz-Credential=minio%2F20220930%2Fus-east-1%2Fs3%2Faws4_request&X-Amz-Date=20220930T081307Z&X-Amz-Expires=3600&X-Amz-SignedHeaders=content-md5%3Bhost&X-Amz-Signature=572a6deb7bec2d9b379f4393fc4982919f704d96e18b11f8e7c02851c762

#### 3.3 Manuelles lunchen des Workflows

Nun öffnet den registrierten Workflow im Flyte Dashboard, [hier](http://localhost:30081/console/projects/flytesnacks/domains/development/workflows/flytesnacks.workflows.workflow_part1.pipeline), und klickt auf `Lunch Workflow`.

Vergewissert euch, dass die Tasks erfolgreich durchgelaufen sind

In [11]:
!kubectl get pods -n flytesnacks-development

NAME                        READY   STATUS      RESTARTS   AGE
a42t7brgtndsmrrbttxj-n0-0   0/1     Completed   0          7m32s
a42t7brgtndsmrrbttxj-n1-0   0/1     Completed   0          6m24s
a42t7brgtndsmrrbttxj-n2-0   0/1     Completed   0          5m24s


Zeigt euch die Logs für den ersten Tasks `n0` an

In [12]:
!kubectl get pods -n flytesnacks-development | grep 'n0' | awk '{print $1}' | xargs -L 1 kubectl -n flytesnacks-development logs 

tar: Removing leading `/' from member names

INFO:root:Data prepared


Zeigt euch die Logs für den zweiten Tasks `n1` an

In [13]:
!kubectl get pods -n flytesnacks-development | grep 'n1' | awk '{print $1}' | xargs -L 1 kubectl -n flytesnacks-development logs 

tar: Removing leading `/' from member names

INFO:root:Model trained and evaluated


Zeigt euch die Logs für den dritten Tasks `n2` an

In [14]:
!kubectl get pods -n flytesnacks-development | grep 'n2' | awk '{print $1}' | xargs -L 1 kubectl -n flytesnacks-development logs 

tar: Removing leading `/' from member names

INFO:root:Model deployed and tested


Logs von einem Kubernetes Pod zeigt man grundsätzlich folgendermaßen an:

```console
kubectl get pods -n <namespace name>   # Finde pod id heraus
kubectl -n <namespace name> logs -f <pod id>
```

### 4 Automatische Workflow Registrierung

#### 4.1 Git Repository initialisieren

In [ ]:
%%bash
if git rev-parse --git-dir > /dev/null 2>&1; then
  : # This is a valid git repository (but the current working
    # directory may not be the top level.
    # Check the output of the git rev-parse command if you care)
    echo "Warning, this notebook assumes one starts from a fresh git repository!"
else
  : 
  git init
  git add .
  git branch -m main
  git commit -m "First commit"
fi

0. Erstellt ein **privates** github repository
1. Substituiert euren GitHub Username für `<YOUR_GITHUB_USER>` unten
2. 
    - Erzeugt einen persönlichen Access Token, hier: https://github.com/settings/tokens/new
    - Gebt die Permissions für Repo und klickt `Generate token`
        - [x] repo
          - [x] repo:status
          - [x] repo_deployment
          - [x] public_repo
          - [x] repo:invite
          - [x] security_events
    - Substituiert euren GitHub Token für `<YOUR_GITHUB_TOKEN>` unten
3. Substituiert eure GitHub Repo unten.
4. Führt die nächste Zelle aus und **entfernt sie anschließend, damit das Token nicht in der Output Zelle des notebooks gespeichert wird!**



**YOUR GITHUB TOKEN IS LIKE A PASSWORD TO YOUR GITHUB ACCOUNT, DO NOT SHARE!**


In [ ]:
%%bash
cat > .env <<EOF
GITHUB_USER=<YOUR_GITHUB_USER>
GITHUB_TOKEN=<YOUR_GITHUB_TOKEN>
GITHUB_REPO=https://github.com/<owner>/<repo>.git
GITHUB_REPO_SSH=git@github.com:<owner>/<repo>.git
EOF

Ladet die Variablen der Environment

In [ ]:
import dotenv
dotenv.load_dotenv()

Pusht den Code auf den main branch euer remote Repository

In [ ]:
!git remote add origin ${GITHUB_REPO_SSH}
!git push -u origin main


#### 4.2 Installiert Argo CD

In [ ]:
!kubectl create ns argo

Installiert Argo CD in den Cluster

In [ ]:
!kubectl apply -n argo -f https://raw.githubusercontent.com/argoproj/argo-workflows/master/manifests/quick-start-postgres.yaml

Wartet bis die Pods gestartet sind

In [ ]:
!kubectl wait --for condition=ready --timeout=300s pods --all -n argo

Überzeugt euch, dass die Pods da sind

In [ ]:
!kubectl get pods -n argo

Macht den Port 2746 lokal verfügbar, indem ihr in einem neuen Terminalfenster den folgenden Befehl ausführt: 
```bash
kubectl -n argo port-forward svc/argo-server 2746:2746
```
Anschließend könnt ihr das `Argo CD Dashboard` auf https://localhost:2746 öffnen

Erzeugt das Kubernetes Secret in `argo_cred.yaml`, damit der Argo Workflow im Cluster auf die GitHub Repository zugreifen kann. Wir substituieren die Environment Variablen mit envsubst:

In [ ]:
!cat argo_cred.yaml | envsubst | kubectl apply -f -

#### 4.3 Erstellt den Argo Workflow (`workflow_part1.py`)
Der folgende Argo Workflow könnte von einem CI/CD Workflow gestartet werden, sodass bei jedem Commit in `flytesnacks/workflows` der Flyte Workflow gestartet wird. So wäre der Stand des Codes mit den Experimenten verknüpft.

Der folgende Befehlt sollte eure GitHub Repo zeigen (HTTPS Link):

In [ ]:
!echo $GITHUB_REPO

Erzeugt den Argo Workflow in `argo_cicd_workflow.yaml`. Indem ihr die Environment Variable `WORKFLOW=workflow_part1` mit envsubst substituiert wird der Flyte Workflow `flytesnacks/workflows/workflow_part1.py` gepackt, registriert und ausgeführt.

**Note** `Argo Workflow` (CD) triggers `Flyte Workflow` (MlOps)

In [ ]:
%env WORKFLOW=workflow_part1
!cat argo_cicd_workflow.yaml | envsubst | kubectl -n argo create -f -

Den Status des CICD workflows kann man unter [https://localhost:2746/workflows?limit=500](https://localhost:2746/workflows?limit=500) sehen.

1. Zunächst wird das Github repository geclont.
2. Dann wird der Flyte workflow registriert und ausgeführt.

Den Status des Flyte workflows kann man in der  [Flyte Console](http://127.0.0.1:30081/console/projects/flytesnacks/domains/development/workflows/flytesnacks.workflows.workflow_part1.pipeline) sehen.

Schaut, dass der Argo Workflow erfolgreich durchläuft:

In [ ]:
!kubectl get workflow -n argo

Schaut, dass der Workflow in Flyte durchläuft, es sollten nacheinander die Pods mit der Endung `-n0-0`, `-n1-0` und `-n2-0` auftauchen:

In [ ]:
!kubectl get pods -n flytesnacks-development

# Session 2

## 5 Mlflow-Tracking-Server Deployment

#### 5.1 Baut das Docker Image für Mlflow

In [ ]:
!docker build -t localhost:5000/mlflow:latest infrastructure/mlflow_tracking_server

In [ ]:
!docker push localhost:5000/mlflow:latest

Deployt Mlflow im Cluster. Dafür wird das Mlflow Image benötigt

In [ ]:
!kubectl apply -f infrastructure/mlflow_tracking_server/manifests/namespace.yaml
!kubectl apply -f infrastructure/mlflow_tracking_server/manifests

In [ ]:
!kubectl wait --for condition=ready --timeout=300s pods --all -n mlflow

Überzeugt euch, dass die Pods da sind

In [ ]:
!kubectl get pods -n mlflow

#### 5.2 Minimales Mlflow Beispiel

**Wichtig:** Macht Mlflow auf dem lokalen Port 5001 verfügbar, indem ihr in einem **neuen Terminalfenster** den **folgenden Befehl ausführt**: 
```bash
kubectl -n mlflow port-forward svc/mlflow-server-service 5001:5000
```
Anschließend öffnet das `Mlflow Dashboard` auf http://127.0.0.1:5001

Führt dieses minimale Mlflow Experiment aus und schaut, dass im `Mlflow Dashboard` ein neuer Run mit Metrik `foo` auftaucht.

In [ ]:
import mlflow
mlflow.set_tracking_uri("http://localhost:5001")
mlflow.start_run()
mlflow.log_metric("foo", 1)
mlflow.end_run()

#### 5.3 Mlflow in der automatisierten Pipeline (`workflow_part2.py`)

Erzeugt den Argo Workflow in `argo_cicd_workflow.yaml`. Indem ihr die Environment Variable `WORKFLOW=workflow_part2` mit envsubst substituiert wird der Flyte Workflow `flytesnacks/workflows/workflow_part2.py` gepackt, registriert und ausgeführt.

**Note** `Argo Workflow` (CD) triggered `Flyte Workflow` (MlOps)

In [ ]:
%env WORKFLOW=workflow_part2
!cat argo_cicd_workflow.yaml | envsubst | kubectl -n argo create -f -

Schaut, dass der Argo Workflow erfolgreich durchläuft:

In [ ]:
!kubectl get workflow -n argo

Schaut, dass der Workflow in Flyte durchläuft, es sollten nacheinander die Pods mit der Endung `-n0-0`, `-n1-0` und `-n2-0` auftauchen:

In [ ]:
!kubectl get pods -n flytesnacks-development

Öffnet das [Mlflow Dashboard](http://127.0.0.1:5001) und schaut, dass ein neuer Run angelegt, entsprechende Parameter und Metriken gelogged und Artifakte registriert worden sind.